# Landmarks Path Computation
In this notebook, we train emotion models using the CK+ dataset.
- Read landmarks files and convert it to a normalized space.
- TODO: PCA to avoid unnecessary landmarks
- Based on the normalized landmarks, compute its patch regresssion.
- 

## Function to read and pre-process features from a file
The dataset contains landmarks computed using AAM. These landmarks are stored as a sequence of (x \n y \n) floats dots. Each frame has its own landmark file. To achieve a normalized feature set, it is set the nose as the reference point and the distance between the eyes as the reference size. For each landmark, the final feature is its distance to the reference point divided by the reference size.

In [ ]:
using Distances
function readLandmarksFile(filename)
    # println("Getting features from: ", filename)
    
    # Database parser
    dots = map((x)->parse(x), split(readall(filename), ['\n', ' '], limit=0, keep=false));
    getLandmarkPoint(id) = [dots[id*2-1], dots[id*2]];
    
    # Reference Point: nose
    referenceID = 34
    referencePoint = getLandmarkPoint(referenceID)
    
    # Reference Size: distance between eyes
    referenceSize = evaluate(Euclidean(), getLandmarkPoint(40), getLandmarkPoint(43))
    if referenceSize < 1.0 error("Distance between eyes less than one.") end
    
    # Normalized vector
    normalizedVector = []
    for id = 1:round(Int,(length(dots) / 2))
        if id == referenceID; continue end
        append!(normalizedVector, (referencePoint - getLandmarkPoint(id)) / referenceSize)
    end
    
    return normalizedVector
end;

## Function to perform patch regression over the landmarks
Using the above function, all the landmarks from a video sequence are read and normalized. Then, for each landmark trajectory, it is computed its patch regression. The regression uses a second order polinomial function: a + bx + cx^2. The resulting vector contains the parameters [x0,xt,a,b,c] for each landmark patch.

In [ ]:
# returns patch_xy[file][landmark*2]
function getPatchFeatures(dir)
    landmarks4frame = []
    for f = readdir("$dir")
        push!(landmarks4frame, readLandmarksFile("$dir/$f"))
    end
    return landmarks4frame
end;

In [ ]:
# returns the (x y) patch for a given landmark
function splitXY(patch, landmark)
    x, y = Float64[], Float64[]
    for file = 1:round(Int, length(patch))
        push!(x, patch[file][landmark*2-1])
        push!(y, patch[file][landmark*2])
    end
    return x, y
end;

In [ ]:
# return abcd (cubic) params for a given (x, y) patch
using LsqFit
function cubicRegression(x, y)
    model(x, p) = p[1] + p[2] * x + p[3] * x .^ 2 + p[4] * x .^ 3;
    fit = curve_fit(model, x, y, [0.,0.,0.,0.]);
    return fit.param
end;
function squareRegression(x, y)
    model(x, p) = p[1] + p[2] * x + p[3] * x .^ 2;
    fit = curve_fit(model, x, y, [0.,0.,0.]);
    return fit.param
end;

In [ ]:
using Gadfly
function drawCubicCurve(x, y, param, title="no title")
    plot(layer(x=x, y=y, Geom.point, order=1),
         layer((x)->param[1] + param[2] * x + param[3] * x .^ 2 + param[4] * x .^ 3, minimum(x), maximum(x)),
         Guide.title(title))
end;
function drawSquareCurve(x, y, param, title="no title")
    plot(layer(x=x, y=y, Geom.point, order=1),
         layer((x)->param[1] + param[2] * x + param[3] * x .^ 2, minimum(x), maximum(x)),
         Guide.title(title))
end;

In [ ]:
# performs regression for each landmark on a video clip
# returns [x0, xt, a, b, c] * lenght(landmarks), i.e. SVM ready
function landmarksRegressionParams(dir, debug=false)
    abc = []
    patch = getPatchFeatures(dir)
    for landmark = 1:floor(Int, length(patch[1]) / 2)
        x, y = splitXY(patch, landmark)
        p = []
        try
            p = squareRegression(x, y)
        catch
            warn("Landmark $landmark: Skipping regression due to an awesome error while performing curve fitting")
            p = [0.,0.,0.]
        finally
            push!(abc, minimum(x))
            push!(abc, maximum(x))
            append!(abc, p)
            if debug display(drawSquareCurve(x, y, p, "landmark: $landmark")) end            
        end
    end
    return abc
end;
# p = landmarksRegressionParams("/home/data/ckplus/Landmarks/S100/001");

In [ ]:
using Gadfly
function drawLandmarksCurves(param)     # TODO - a not so hardcoded revision
    layers = Array{Gadfly.Layer,1}[]
    for i in 1:floor(Int, length(param) / 5)
        x0, xt, a, b, c = param[i*5-4], param[i*5-3], param[i*5-2], param[i*5-1], param[i*5]
        push!(layers, layer((x)->a+b*x+c*x.^2, x0, xt))
    end
    plot( layers[1], layers[2], layers[3], layers[4], layers[5], layers[6], layers[7], layers[8], layers[9], layers[10], layers[11], layers[12], layers[13], layers[14], layers[15], layers[16], layers[17], layers[18], layers[19], layers[20], layers[21], layers[22], layers[23], layers[24], layers[25], layers[26], layers[27], layers[28], layers[29], layers[30], layers[31], layers[32], layers[33], layers[34], layers[35], layers[36], layers[37], layers[38], layers[39], layers[40], layers[41], layers[42], layers[43], layers[44], layers[45], layers[46], layers[47], layers[48], layers[49], layers[50], layers[51], layers[52], layers[53], layers[54], layers[55], layers[56], layers[57], layers[58], layers[59], layers[60], layers[61], layers[62], layers[63], layers[64], layers[65], layers[66], layers[67], Guide.title("Facial Landmarks Patchs"))
end;
# drawLandmarksCurves(p)

# Function to iterate over the CK+ dataset
For each sample, read the landmarks and the emotion result files. Using the above functions, the landmarks are transformed to an array of regression parameters. To perform the SVM trainning, the trainning matrix is composed by the regression parameters, where each row represent a sample video. In the same manner, the result matrix (for trainning) is composed by the emotion ID for the respective sample in the trainning matrix. So, for SVM trainning there are two matrixes, MTrainning: samples x (landmarks * regression_params), MTResults: samples x 1.

In [ ]:
# read the sample file containing Emotion or FACS codes
function readResultFile(dir)
    results = []
    if !ispath(dir)
        return results
    end
    for a in readdir(dir)
        filename = "$dir/$a"
        append!(results, map((x)->convert(Int, parse(x)), split(readall(filename), ['\n', ' '], limit=0, keep=false)))
    end
    return(results)
end;
# readResultFile("/home/data/ckplus/Emotion/S005/001")

In [43]:
# iterate over all samples to mount the trainning matrix for svm: X <samples> x <landmarks * regression_params>
# returns also the path "$subject/$sample" referencing each sample in X
function loadTrainningSamples(dir)
    landmarksDir = "$dir/Landmarks"
    X, Path = Matrix, []
    first, c = true, 0
    for subjectDir = readdir(landmarksDir), sampleDir = readdir("$landmarksDir/$subjectDir")
        c += 1
        if (c % 100 == 0) println("$c") end
        row = []
        try
            row = landmarksRegressionParams("$landmarksDir/$subjectDir/$sampleDir")
        catch
            warn("$c: Skipping due to some bizarre error while loading landmarks regression parameters.")
            continue
        end
        if !first
            X = vcat(X, reshape(row, 1, size(row)[1]))
        else
            X = reshape(row, 1, size(row)[1])
            first = false
        end
        push!(Path, "$subjectDir/$sampleDir")
    end
    return X, Path
end
# @time S, P = loadTrainningSamples("/home/data/ckplus");
# drawLandmarksCurves(S[100,:])

loadTrainningSamples (generic function with 1 method)

In [ ]:
# todo: support facs
function parseSVM_XY(dir, samples, paths, key, debug = false)
    X, Y = Matrix(0,size(samples)[2]) , []
    c = 0
    for p in paths
        c += 1
        if debug 
            if c > 10 break end
            println("--- $c: $p ---")
        end
        results = readResultFile("$dir/$p")
        if length(results) > 0
            for r in results
                ry = (r == key)? 1: -1
                X = vcat(X, samples[c,:])
                push!(Y, ry)
                if debug println("$c: $r -> $ry") end
            end
        else
            X = vcat(X, samples[c,:])
            push!(Y, -1)
            if debug println("$c: nothing -> -1") end
        end
    end
    return X, Y
end
# X, Y = parseSVM_XY("/home/data/ckplus/Emotion", S, P, 6, true)

In [ ]:
using RegERMs
function trainCKPlus(dir, debug = false)
    # landmarksDir = "$dir/Landmarks
    # emotionDir = "$dir/Emotion"
    # facsDir =  "$dir/FACS"
    
    # Train Emotion
    S, P = loadTrainningSamples());
    
end

trainCKPlus("/home/data/ckplus", true)

In [ ]:
a = Matrix(0,size(S)[2])
S[1,:]

In [ ]:
vcat(a, S[1,:])

In [ ]:
size(S)[2]